Источник: https://github.com/PacktPublishing/Hands-On-Image-Processing-with-Python/tree/master

In [107]:
# !pip install -U scikit-image

In [108]:
import os
from PIL import Image, ImageFilter
import numpy as np
from numpy import asarray
import shutil
from scipy.ndimage import gaussian_filter
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from numpy import linalg as LA
from skimage import exposure, feature
import warnings
warnings.filterwarnings("ignore")

In [109]:
# constants

# 1
P = 256

# 4
K = 5

In [110]:
# functions
def remove(path):
    if os.path.exists(path):  
        if os.path.isfile(path) or os.path.islink(path):
            os.unlink(path)
        else:
            shutil.rmtree(path)

def contrast_stretching(img: np.array) -> np.array:
    return ((img - img.min()) / (img.max() - img.min()) * (P-1)).astype(np.uint8)

In [111]:
for i in range(1, 9):
    remove(f"files/{i}")
for i in range(1, 9):
    os.mkdir(f"files/{i}")


img = Image.open("files/image.jpg")

### 1. Загрузка и визуализация изображения
#### a) Числовые характеристики изображения

In [112]:
print(f"Размерность изображения: {img.size[0]}x{img.size[1]}")
print(f"Число световых каналов: {len(img.getbands())}")
print(f"Яркостное разрешение: {len(img.getbands()) * 8} bits")


Размерность изображения: 1600x1060
Число световых каналов: 3
Яркостное разрешение: 24 bits


#### б) Если изображение содержит много цветовых каналов, то привести его к одноканальному (в градациях серого) в диапазоне [0; P], P = 255 (яркостное разрешение – 1 байт) 

In [113]:
gray = img.convert('L')
gray.save('files/greyscale.jpg')

img_array = asarray(gray)

### 2. Градиентные операторы

#### a) Рассчитать градиентные изображения в вертикальном и горизонтальном направлениях, а также изображения модулей и углов градиентов, используя оператор Робертса
Источник: https://en.wikipedia.org/wiki/prewitt_cross

Источник: https://habr.com/ru/articles/114452/

In [114]:
roberts_cross_v = np.array( [[1, 0 ], 
                             [0,-1 ]] ) 
  
roberts_cross_h = np.array( [[ 0, 1 ], 
                             [ -1, 0 ]] )

g_vertical = cv2.filter2D(img_array, -1, roberts_cross_v)
exp_g_vertical = cv2.equalizeHist(g_vertical)
Image.fromarray(exp_g_vertical).convert('L').save('files/2/roberts_vertical.jpg')

g_horizontal = cv2.filter2D(img_array, -1, roberts_cross_h) 
exp_g_horizontal = cv2.equalizeHist(g_horizontal)
Image.fromarray(exp_g_horizontal).convert('L').save('files/2/roberts_horizontal.jpg')

g_module = np.sqrt( np.square(g_horizontal) + np.square(g_vertical)).astype(np.uint8)
exp_g_module = cv2.equalizeHist(g_module)
Image.fromarray(exp_g_module).convert('L').save('files/2/roberts_module.jpg')

g_angle = (np.arctan(g_vertical/g_horizontal) - 0.75 * np.pi).astype(np.uint8)
exp_g_angle = cv2.equalizeHist(g_angle)
Image.fromarray(exp_g_angle).convert('L').save('files/2/roberts_angle.jpg')

### б) Рассчитать градиентные изображения в вертикальном и горизонтальном направлениях, а также изображения модулей и углов градиентов, используя оператор Превитта
Источник: https://en.wikipedia.org/wiki/Prewitt_operator

In [115]:
prewitt_cross_v = np.array( [[1, 0, -1 ], 
                             [1, 0, -1],
                             [1, 0, -1]]) 
  
prewitt_cross_h = np.array( [[1, 1, 1 ], 
                             [0, 0, 0],
                             [-1, -1, -1]])

g_vertical = cv2.filter2D(img_array, -1, prewitt_cross_v)
exp_g_vertical = cv2.equalizeHist(g_vertical)
Image.fromarray(exp_g_vertical).convert('L').save('files/2/prewitt_vertical.jpg')

g_horizontal = cv2.filter2D(img_array, -1, prewitt_cross_h) 
exp_g_horizontal = cv2.equalizeHist(g_horizontal)
Image.fromarray(exp_g_horizontal).convert('L').save('files/2/prewitt_horizontal.jpg')

g_module = np.sqrt(np.square(g_horizontal) + np.square(g_vertical)).astype(np.uint8)
exp_g_module = cv2.equalizeHist(g_module)
Image.fromarray(exp_g_module).convert('L').save('files/2/prewitt_module.jpg')

g_angle = np.arctan(g_vertical/g_horizontal).astype(np.uint8)
exp_g_angle = cv2.equalizeHist(g_angle)
Image.fromarray(exp_g_angle).convert('L').save('files/2/prewitt_angle.jpg')

### в) Рассчитать градиентные изображения в вертикальном и горизонтальном направлениях, а также изображения модулей и углов градиентов, используя оператор Собеля
Источник: https://en.wikipedia.org/wiki/Sobel_operator

In [116]:
sobel_cross_v = np.array( [[1, 0, -1 ], 
                             [2, 0, -2],
                             [1, 0, -1]]) 
  
sobel_cross_h = np.array( [[1, 2, 1 ], 
                             [0, 0, 0],
                             [-1, -2, -1]])

g_vertical = cv2.filter2D(img_array, -1, sobel_cross_v)
exp_g_vertical = cv2.equalizeHist(g_vertical)
Image.fromarray(exp_g_vertical).convert('L').save('files/2/sobel_vertical.jpg')

g_horizontal = cv2.filter2D(img_array, -1, sobel_cross_h) 
exp_g_horizontal = cv2.equalizeHist(g_horizontal)
Image.fromarray(exp_g_horizontal).convert('L').save('files/2/sobel_horizontal.jpg')

g_module = np.sqrt(np.square(g_horizontal) + np.square(g_vertical)).astype(np.uint8)
exp_g_module = cv2.equalizeHist(g_module)
Image.fromarray(exp_g_module).convert('L').save('files/2/sobel_module.jpg')

g_angle = np.arctan(g_vertical/g_horizontal).astype(np.uint8)
exp_g_angle = cv2.equalizeHist(g_angle)
Image.fromarray(exp_g_angle).convert('L').save('files/2/sobel_angle.jpg')

### 3) Применить к изображению фильтр Гаусса с различными значениями среднеквадратического отклонения sigma:

In [117]:
for sigma in (0.5, 5, 10, 20):
    image = gray.filter(ImageFilter.GaussianBlur(sigma))
    image.save(f"files/3/gaussian_filter_sigma_{sigma}.jpg")

### 4. Применить оператор Собеля к изображению, сглаженному фильтром Гаусса с различными значениями среднеквадратического отклонения sigma.

In [118]:
for sigma in (0.5, 5, 10, 20):
    sobel_gauss_module = Image.fromarray(exp_g_module).filter(ImageFilter.GaussianBlur(sigma))
    sobel_gauss_module.convert('L').save(f'files/4/sobel_gauss_module_sigma_{sigma}.jpg')
    sobel_gauss_angle = Image.fromarray(exp_g_angle).filter(ImageFilter.GaussianBlur(sigma))
    sobel_gauss_angle.convert('L').save(f'files/4/sobel_gauss_angle_sigma_{sigma}.jpg')

### 5) Применить к изображению фильтр лапласиана гауссиан (LoG) с различными размером окна k и значениями среднеквадратического отклонения sigma:
Источник: https://math.stackexchange.com/questions/2445994/discrete-laplacian-of-gaussian-log

Источник: https://homepages.inf.ed.ac.uk/rbf/HIPR2/zeros.htm

#### а) Ядро k = 5, sigma = 0.2

In [119]:
# Проверка для смены типа
np.array_equal(img_array, img_array.astype(np.int16))

True

In [120]:
def LOG(sigma, x, y):
    laplace = -1/(np.pi*sigma**4)*(1-(x**2+y**2)/(2*sigma**2))*np.exp(-(x**2+y**2)/(2*sigma**2))
    return laplace

def LOG_discrete(sigma, n):
    l = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            l[i,j] = LOG(sigma, (i-(n-1)/2),(j-(n-1)/2))
    return l

In [121]:
def LOG_process(k, sigma):
    results = ndimage.convolve(img_array.astype(np.int16), LOG_discrete(sigma, k))

    binary_results = results.copy()
    binary_results[binary_results > 0] = 1
    binary_results[binary_results < 0] = -1

    zero_crossing_results = results.copy()
    zero_crossing_results[zero_crossing_results > 0] = -1
    zero_crossing_results[zero_crossing_results < 0] = -1
    zero_crossing_results[zero_crossing_results == 0] = 0
    return results, binary_results, zero_crossing_results

In [122]:
k, sigma = 5, 0.2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [123]:
k, sigma = 10, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [124]:
k, sigma = 30, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [125]:
k, sigma = 30, 10
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

### 6. Применить к изображению фильтр разности гауссиан (DoG) с различными параметрами sigma1 и alpha = sigma1 / sigma2
##### Источник: https://gist.github.com/leonidk/8798fdbf38db120b8536d25ea2f8c3b4

In [126]:
def DOG(img, sigma1, alpha):
    sigma2 = sigma1 / alpha
    s1 = gaussian_filter(img, sigma1)
    s2 = gaussian_filter(img, sigma2)

    dog = s1 - s2
    return dog

In [127]:
Image.fromarray(DOG(img_array, 2, 1.6)).save(f"files/6/dog_sigma1_{2}_alpha_{1.6}.jpg")
Image.fromarray(DOG(img_array, 5, 1.6)).save(f"files/6/dog_sigma1_{5}_sigma_{1.6}.jpg")
Image.fromarray(DOG(img_array, 2, 5)).save(f"files/6/dog_sigma1_{2}_sigma_{5}.jpg")
Image.fromarray(DOG(img_array, 2, 10)).save(f"files/6/dog_sigma1_{2}_sigma_{10}.jpg")

### 7. Выделить границы на изображении

#### а) Детектор Марра-Хилдрета

##### https://github.com/adl1995/edge-detectors/blob/master/marr-hildreth-edge.py

#### б) Детектор Кэнни
Источник: https://github.com/PacktPublishing/Hands-On-Image-Processing-with-Python/blob/master/Chapter05/Chapter5.ipynb

Минута на видео: 45:00

In [128]:
sigma, low_threshold, high_threshold = 0.5, 200, 255

edges = feature.canny(img_array, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold).astype(np.uint8)
Image.fromarray(contrast_stretching(edges)).save(f"files/7/canny_sigma_{sigma}.jpg")

In [129]:
sigma, low_threshold, high_threshold = 2, 50, 200

edges = feature.canny(img_array, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold).astype(np.uint8)
Image.fromarray(contrast_stretching(edges)).save(f"files/7/canny_sigma_{sigma}.jpg")

In [130]:
sigma, low_threshold, high_threshold = 5, 100, 255

edges = feature.canny(img_array, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold).astype(np.uint8)
Image.fromarray(contrast_stretching(edges)).save(f"files/7/canny_sigma_{sigma}.jpg")

In [131]:
sigma, low_threshold, high_threshold = 10, 100, 255

edges = feature.canny(img_array, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold).astype(np.uint8)
Image.fromarray(contrast_stretching(edges)).save(f"files/7/canny_sigma_{sigma}.jpg")

### 8. Провести нерезкое маскирование изображения

#### а) Без использования порога (t = 0)
##### Источник: 
##### Минута на видео: 1:53:40

In [132]:
for (sigma, lambdaa) in [(2, 2), (20, 2), (2, 10), (20, 10)]:
    gauss = asarray(gray.filter(ImageFilter.GaussianBlur(sigma)))
    unsharp = img_array + lambdaa * (img_array - gauss)
    Image.fromarray(unsharp).save(f"files/8/unsharp_mask_without_t_sigma_{sigma}_lambda_{lambdaa}.jpg")    

#### б) С использованием порога (t > 0)

In [137]:
for (sigma, lambdaa, t) in [(2, 2, 100), (20, 2, 100), (2, 10, 100), (20, 10, 100)]:
    gauss = asarray(gray.filter(ImageFilter.GaussianBlur(sigma)))
    unsharp = np.where(np.abs(img_array - unsharp) > t, img_array + lambdaa * (img_array - gauss), gauss)
    Image.fromarray(unsharp).save(f"files/8/unsharp_mask_sigma_{sigma}_lambda_{lambdaa}_t_{t}.jpg")    